<a href="https://colab.research.google.com/github/etuckerman/SOCOTEC/blob/main/SOCOTEC_GPU_elliot_tuckerman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read)

In [13]:
# Check for GPU availability
!nvidia-smi

Mon Dec  9 18:29:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              53W / 400W |  10257MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [14]:
# Install required libraries
!pip install -q transformers torch accelerate datasets openai
!pip install -q bitsandbytes

import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import re

In [22]:
# Implement function library as specified
def add(a, b): return a + b
def square(a): return a ** 2
def cube(a): return a ** 3
def greet(name): return f"Hello, {name}!"

# Function mapping dictionary
FUNCTION_MAPPING = {
    'add': add,
    'square': square,
    'cube': cube,
    'greet': greet
}

In [23]:
# Create synthetic training dataset
training_data = [
    {"input": "What is the sum of 1 and 2?", "output": "add(1, 2)"}
]

# Additional diverse prompts
# Comprehensive test cases covering all functions
test_cases = training_data + [
    # Organized test cases with diverse scenarios
    # Basic addition scenarios
    {"input": "What is the sum of 7 and 3?", "output": "add(7, 3)"},
    {"input": "Add 10 and 15", "output": "add(10, 15)"},
    {"input": "How much is 20 plus 22?", "output": "add(20, 22)"},
    {"input": "Add 50 and 30", "output": "add(50, 30)"},
    {"input": "What is 18 plus 27?", "output": "add(18, 27)"},

    # Square function scenarios
    {"input": "Calculate the square of 5", "output": "square(5)"},
    {"input": "Square of 6", "output": "square(6)"},
    {"input": "Square 7", "output": "square(7)"},
    {"input": "Square of 4", "output": "square(4)"},
    {"input": "Square of 8", "output": "square(8)"},

    # Cube function scenarios
    {"input": "What's the cube of 2?", "output": "cube(2)"},
    {"input": "Cube the number 4", "output": "cube(4)"},
    {"input": "Cube of 3", "output": "cube(3)"},
    {"input": "Cube 5", "output": "cube(5)"},
    {"input": "Cube of 6", "output": "cube(6)"},

    # Greeting scenarios
    {"input": "Greet John", "output": "greet('John')"},
    {"input": "Say hello to Sarah", "output": "greet('Sarah')"},
    {"input": "Greet Emily", "output": "greet('Emily')"},
    {"input": "Greet Michael", "output": "greet('Michael')"},

    # BONUS: Multi-function scenarios (to demonstrate potential)
    {"input": "Add the square of 3 to 10", "output": "add(square(3), 10)"},
    {"input": "Greet the person who got the cube of 2", "output": "greet('cube(2)')"}
]

In [27]:
def format_prompt(input_text):
    return f"""You are an expert at converting natural language to precise Python function calls using ONLY these predefined functions:
- add(a, b): Returns the sum of a and b
- square(a): Returns a squared
- cube(a): Returns a cubed
- greet(name): Returns a greeting for the given name

IMPORTANT: ONLY use these exact function names and signatures.

Convert the following input to EXACTLY the correct function call:
Input: {input_text}
Correct Function Call: """

def generate_function_call(model, tokenizer, input_text, max_length=150):
    prompt = format_prompt(input_text)

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.1  # Very low temperature for more precise output
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # More robust extraction of function call
    function_call_match = re.search(r'Correct Function Call:\s*(.+)', generated_text)

    if function_call_match:
        function_call = function_call_match.group(1).strip()

        # Standardize quotes and prevent generic Python operations
        function_call = function_call.replace('"', "'")

        # Prevent generic print or math operations
        if not re.match(r'^(add|square|cube|greet)\(', function_call):
            return None

        return function_call

    return None

def execute_function_call(function_call):
    try:
        match = re.match(r'(\w+)\((.*?)\)', function_call)
        if not match:
            return "Invalid function call format"

        func_name, args_str = match.groups()

        # More robust argument parsing
        args = []
        for arg in re.findall(r"'[^']*'|\"[^\"]*\"|[^,\s]+", args_str):
            arg = arg.strip("'\"")
            try:
                arg = int(arg) if arg.isdigit() else arg
            except ValueError:
                pass
            args.append(arg)

        if func_name in FUNCTION_MAPPING:
            return FUNCTION_MAPPING[func_name](*args)
        else:
            return f"Unsupported function: {func_name}"

    except Exception as e:
        return f"Error executing function: {str(e)}"

def evaluate_model(model, tokenizer, test_cases):
    results = []
    for case in test_cases:
        function_call = generate_function_call(model, tokenizer, case['input'])
        expected_call = case['output']

        result = {
            'input': case['input'],
            'generated_call': function_call,
            'expected_call': expected_call,
            'call_match': function_call == expected_call
        }

        if result['call_match']:
            try:
                result['execution_result'] = execute_function_call(function_call)
            except Exception as e:
                result['execution_result'] = str(e)

        results.append(result)

    return results

In [19]:
# Model setup
model_name = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_8bit=True,
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
# Run evaluation
evaluation_results = evaluate_model(model, tokenizer, test_cases)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_tok

In [29]:
# Print results
for result in evaluation_results:
    print(f"Input: {result['input']}")
    print(f"Generated Call: {result['generated_call']}")
    print(f"Expected Call: {result['expected_call']}")
    print(f"Call Match: {result['call_match']}")
    if result.get('execution_result'):
        print(f"Execution Result: {result['execution_result']}")
    print("---")

Input: What is the sum of 1 and 2?
Generated Call: add(1, 2)
Expected Call: add(1, 2)
Call Match: True
Execution Result: 3
---
Input: What is the sum of 7 and 3?
Generated Call: add(7, 3)
Expected Call: add(7, 3)
Call Match: True
Execution Result: 10
---
Input: Add 10 and 15
Generated Call: add(10, 15)
Expected Call: add(10, 15)
Call Match: True
Execution Result: 25
---
Input: How much is 20 plus 22?
Generated Call: None
Expected Call: add(20, 22)
Call Match: False
---
Input: Add 50 and 30
Generated Call: add(50, 30)
Expected Call: add(50, 30)
Call Match: True
Execution Result: 80
---
Input: What is 18 plus 27?
Generated Call: add(18, 27)
Expected Call: add(18, 27)
Call Match: True
Execution Result: 45
---
Input: Calculate the square of 5
Generated Call: square(5)
Expected Call: square(5)
Call Match: True
Execution Result: 25
---
Input: Square of 6
Generated Call: square(6)
Expected Call: square(6)
Call Match: True
Execution Result: 36
---
Input: Square 7
Generated Call: square(7)
Expe